케라스의 [tf.keras.datasets.boston_housing](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/boston_housing)로부터 보스턴 주택가격 데이터셋을 불러오겠습니다.
- 1970년대 보스턴 교외의 타운별 데이터 506개
- 각 행은 타운별 데이터
- 각 열은 주택 가격에 영향을 미치는 요소 13개
- 훈련용 404개, 테스트용 102개
- 라벨은 주택가격 중앙값 (단위 : 천달러)  
![](https://drive.google.com/thumbnail?id=1GkxuiFIvsL_k0u5iYNmWsF6oaOG3Ed4P&sz=s4000)  

# 보스턴 주택가격 데이터셋

In [ ]:
from tensorflow import keras
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(f"훈련용 : {train_data.shape}")
print(f"테스트용 : {test_data.shape}")

주택 가격에 영향을 미치는 13개의 요소는 다음과 같습니다.
- CRIM : per capita crime rate by town (범죄율)
- ZN : proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS : proportion of non-retail business acres per town.
- CHAS : Charles River dummy variable (1 if tract bounds river; 0 otherwise) (강변)
- NOX : nitric oxides concentration (parts per 10 million)
- RM : average number of rooms per dwelling (평균 방 개수)
- AGE : proportion of owner-occupied units built prior to 1940 (노후주택 비율)
- DIS : weighted distances to five Boston employment centres
- RAD : index of accessibility to radial highways
- TAX : full-value property-tax rate per $10,000 (재산세 세율)
- PTRATIO : pupil-teacher ratio by town (학생/교사 비율)
- B : 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT : lower status of the population (하위계층 비율)

In [ ]:
columns = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']

파이썬데이터분석 시간에 배운 판다스를 이용해 훈련 데이터를 출력해보겠습니다.

In [ ]:
import pandas as pd

df = pd.DataFrame(train_data, columns=columns)
df

라벨을 마지막 열에 추가하겠습니다.  
타운별 주택가격 중앙값입니다.  
단위는 천달러입니다.

In [ ]:
df['PRICE'] = train_targets
df

기본적인 통계정보를 출력해보겠습니다.

In [ ]:
train_stats = df.describe()
train_stats.transpose()

seaborn의 [pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html)을 사용해서 몇몇 조합의의 산포도를 그려보겠습니다.  
주택가격 예측이 목적이므로 맨 오른쪽 열을 순서대로 해석해보죠.
- 대부분 범죄율(CRIM)이 낮지만 왼쪽 위에 점이 몇개 찍혀 있습니다. 우범지대겠지요. 집값(PRICE)이 낮습니다.
- 방 개수(RM)는 집값(PRICE)과 비례합니다.
- 하위계층비율(LSTAT)은 집값과 반비례합니다.
- 집값(PRICE)의 히스토그램입니다.

In [ ]:
import seaborn as sns
sns.pairplot(df[["CRIM","RM","LSTAT","PRICE"]])

**[실습1] (5분) 테스트 데이터에 라벨을 추가한 데이터 프레임을 출력하시오.**

각 특성들은 스케일이 매우 다릅니다.  
학습시 스케일이 작은 특성은 무시되고 스케일이 큰 특성은 지배적이 됩니다.  
결과적으로는 적은 정보를 지닌 데이터로 학습시키는 셈이 됩니다.  
이를 방지하기 위해 데이터를 특성별로 정규화해야합니다.  
주의할 점은 테스트 데이터를 정규화할 때 훈련 데이터의 평균과 표준편차를 사용한다는 점입니다.

In [ ]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

# 회귀

우리가 여태까지 공부한 문제는 분류(classification)문제입니다.  
시험으로 치면 객관식 문제를 푸는 셈이지요.  
MNIST, Fashion MNIST는 10지선다 문제를 푸는 셈이고 로이터 뉴스는 46지선다 문제를 푸는 셈이고 IMDB는 OX 문제를 푸는 셈입니다.  
분류문제에서는 통상적으로 교차 엔트로피(cross entropy)를 손실함수로 사용합니다. 또한, 평가지표로는 정확도(accuracy)를 사용합니다.  
![](https://drive.google.com/thumbnail?id=1wdEkIUaKqrAh76qYB41eKUkEKHergDy0&sz=s4000)

---
회귀(regression)문제는 시험으로 치면 주관식 문제를 푸는 셈입니다.  
내일 더울지 추울지 예측하는 문제는 분류문제이고 내일 기온을 예측하는 문제는 회귀문제입니다.  
회귀문제에서는 평가지표로 정확도를 사용해서는 안됩니다.  
내일 기온을 칼같이 예측하기는 어려울테니까요.  
회귀문제에서는 평가지표로 평균절대오차(Mean Absolute Error)를 사용합니다.  
두 벡터 $(x_1,x_2,\cdots,x_n)$, $(y_1,y_2,\cdots,y_n)$의 평균절대오차는 이름 그대로 오차의 절대값의 평균 ${\rm MAE} = {1 \over n}\sum_{i=1}^n |y_i-x_i|$으로 정의합니다.  
평가시 평균절대오차는 작을수록 좋습니다 (정확도는 클수록 좋았죠).  
손실함수는 통상적으로 평균제곱오차(Mean Squared Error)를 사용합니다.  
두 벡터 $(x_1,x_2,\cdots,x_n)$, $(y_1,y_2,\cdots,y_n)$의 평균제곱오차는 이름 그대로 오차의 제곱의 평균 ${\rm MSE} = {1 \over n}\sum_{i=1}^n (y_i-x_i)^2$으로 정의합니다.  
평균제곱오차는 [딥러닝I](https://youtu.be/Bj4TS9ip9SM?t=897)에서 등장은 했는데 실제로는 교차엔트로피만 사용해왔죠.  
![](https://drive.google.com/thumbnail?id=1kDAhPbUz7_uNGF_sc1HEGcB0EzAd87ik&sz=s4000)

---
예를 들어, $(1,2,3,4,5)$와 $(3,3,3,3,3)$의 평균절대오차는
$$
{\rm MAE} = {1 \over 5}(|1-3|+|2-3|+|3-3|+|4-3|+|5-3|) = {1 \over 5}(2+1+0+1+2) = {6 \over 5}
$$
이고 평균제곱오차는
$$
{\rm MSE} = {1 \over 5}((1-3)^2+(2-3)^2+(3-3)^2+(4-3)^2+(5-3)^2) = {1 \over 5}(4+1+0+1+4) = 2
$$
입니다.  
이 예에서 보다시피 분산이 평균분포와의 평균제곱오차입니다.

---
신경망을 다음과 같이 구성하겠습니다.  
입력 뉴런의 수는 데이터의 특성수이고 출력 뉴런의 수는 1입니다.  
이진분류에서는 마지막층에서 시그모이드 변환을 했지만 회귀에서는 통상적으로 아무것도 하지 않습니다.  
이진분류에서는 확률을 출력해야 하니까 시그모이드 변환을 했지만 주택가격은 0가 1사이에 있을 필요가 없으니까요.  
[compile](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#compile) 메서드에서 `loss=["mse"]`로 손실함수를 평균제곱오차로 설정하고 `metrics="mae"`로 평가지표를 평균절대오차로 설정합니다.  
앞으로 신경망을 반복해서 초기화할거라서 아예 신경망을 생성하는 함수를 만들어 놓겠습니다.  
![](https://drive.google.com/thumbnail?id=1di1n97y6s1Fbsr-B-9wl-m1onhdlt_w6&sz=s4000)

---
신경망의 출력값이 $x_1$이고 라벨이 $y_1$이라면 평균제곱오차는 $(x_1-y_1)^2$이고 평균절대오차는 $|x_1-y_2|$입니다.  
$n$개의 배치묶음이 입력되어 출력값이 $x_1,x_2,\cdots,x_n$이고 라벨이 $y_1,\cdots,y_n$이라면 평균을 하게 되어 ${1 \over n}\sum_{i=1}^n (y_i-x_i)^2$와 ${1 \over n}\sum_{i=1}^n |y_i-x_i|$이 나옵니다.  
다시말해 배치묶음을 입력하면 두 벡터 $(x_1,x_2,\cdots,x_n)$, $(y_1,y_2,\cdots,y_n)$의 평균제곱오차와 평균절대오차가 됩니다.

---
입력데이터는 파란색입니다.  
이를 입력받아서 역전파를 통해 가급적 빨간색과 비슷한 값을 신경망이 출력할 수 있도록 파라미터들을 업데이트합니다.  
![](https://drive.google.com/thumbnail?id=1VytfDs5vZowI7e6GjRX40ZPPqrMvHTg4&sz=s4000)

In [ ]:
from keras import models
from keras.layers import Dense
from tensorflow.keras.utils import plot_model

def build_model():
    model = keras.Sequential([
        Dense(64, input_shape=(13,), activation="relu"),
        Dense(64, activation="relu"),
        Dense(1)
    ])
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    return model

model=build_model()
plot_model(model, show_shapes=True, show_layer_activations=True)

# K-겹 검증 (K-fold Validation)

훈련 데이터가 404개 밖에 없습니다.  
하이퍼 파라미터 탐색을 위해 훈련데이터와 검증데이터로 분할해야 하는데 데이터가 적어서 검증데이터에 적은 양을 할당할 수 밖에 없습니다.  
적은 양때문에 샘플링 편차가 커서 검증 결과를 믿을 수가 없습니다.  
이럴때 쓰는 방법이 K-겹 검증입니다.  
훈련 데이터를 K-등분한 후 하나를 검증데이터로 사용합니다.  
이렇게 얻은 K개의 검증결과를 평균합니다.  

![](https://drive.google.com/thumbnail?id=1N9g6DhhKpj0WjzzIj1ZDRN7YmZAj8UVm&sz=s4000)

---
4겹 검증 코드입니다.  
훈련 데이터의 개수가 404개이므로 `num_val_samples`는 101입니다.  
훈련데이터를 4등분해서 그중에 i번째 블럭(그림에서 파란 부분)을 뽑아내서 `val_data`라고 둡니다.  
i번째 블럭의 왼쪽 블럭과 오른쪽 블럭(그림에서 녹색부분)을 `np.concatenate`를 이용해 합쳐서 `partial_train_data`를 만듭니다.  
행렬 2개를 합치는 것이므로 옆으로 붙일지 위아래로 붙일지를 명시해야 합니다.  
위아래로 붙여야 하므로 `axis=0`으로 설정합니다.  
위에서 정의한 함수로 모델을 만든후 `partial_train_data`로 훈련시킵니다.  
`val_data`로 성능을 평가해 평균절대오차를 `all_scores`에 추가합니다.

In [ ]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]])
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=16, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

샘플링 수가 적어서 편차가 꽤 되네요.

In [ ]:
print(all_scores)

넷을 평균하면 더 신뢰할 수 있습니다.  
K-겹 검증을 하는 이유입니다.

In [ ]:
print(np.mean(all_scores))

본격적으로 K-겹 검증을 사용해서 최적의 학습회수를 찾겠습니다.  
오랫동안 훈련시켜서 과적합을 일으킵니다.  
한참 걸리니 잠시 쉬었다 오세요.  
`mae_history`는 에퍽이 끝날때마다 검증데이터로 측정한 평균절대오차들을 모아놓은 리스트
$$
[a_1, a_2, \cdots, a_{500}]
$$
입니다.  
모델이 4개이므로 `mae_history`도 4개가 나올텐데 `all_mae_histories`는 이들로 이루어진 리스트
$$
[[a_1, a_2, \cdots, a_{500}], [b_1, b_2, \cdots, b_{500}], [c_1, c_2, \cdots, c_{500}], [d_1, d_2, \cdots, d_{500}]]
$$
입니다.

In [ ]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]])
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=16, verbose=0)
    mae_history = history.history["val_mae"]
    all_mae_histories.append(mae_history)

각 에퍽별로 4개의 평균절대오차를 평균합니다.  
`average_mae_history`는 에퍽이 끝날때마다 측정한 4개 모델의 평균절대오차들의 평균을 모아놓은 리스트
$$
[{a_1+b_1+c_1+d_1 \over 4}, {a_2+b_2+c_2+d_2 \over 4}, \cdots, {a_{500}+b_{500}+c_{500}+d_{500} \over 4}]
$$
입니다.

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

그래프를 그려보면 초반 평균절대오차가 너무 커서 언제부터 과적합이 일어나는지 눈으로 확인이 어렵습니다.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

지나치게 큰 초반 10에퍽은 제외하겠습니다.  
변동이 크지만 대략 130에퍽부터 과적합이 시작되네요.

In [ ]:
truncated_mae_history = average_mae_history[10:]
plt.plot(range(1, len(truncated_mae_history) + 1), truncated_mae_history)
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.show()

# 학습, 평가, 예측

훈련 데이터 전체를 사용해 130에퍽동안 훈련시키겠습니다.

In [ ]:
model = build_model()
model.fit(train_data, train_targets,
          epochs=130, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

학습할때마다 달라지지만 테스트 데이터로 확인한 최종 평균절대오차는 대략 2,600달러 정도네요.

In [ ]:
test_mae_score

첫번째 테스트 데이터에 대한 예측가격과 실제가격입니다.  
학습할때마다 오차는 달라집니다.

In [ ]:
predictions = model.predict(test_data)
print(f"예측가격 : {predictions[0]}")
print(f"실제가격 : {test_targets[0]}")

**[실습2] (5분) 1번 문제에서 구한 데이터프레임에 신경망의 예측값을 추가해서 출력하시오.**

**[실습3] (5분) 실제가격과 예측가격의 산포도를 그리시오. $y=x$의 그래프도 그려넣으시오.**

**[실습4] (5분) (예측가격 - 실제가격)의 히스토그램을 그리시오.**

**[실습5] (10분) (i) 앞 8개 특성만 사용해서 훈련시킨후 평가하시오.**

**(ii) 앞 4개 특성만 사용해서 훈련시킨후 평가하시오.**

**[과제1] 표준정규분포를 따라 랜덤하게 k개의 노이즈 특성을 만들어 훈련데이터에 추가하시오. 표준정규분포를 따라 랜덤하게 k개의 노이즈 특성을 만들어 테스트 데이터에 추가하시오. 13+k의 특성으로 신경망을 학습시킨후 테스트 데이터로 평균절대오차를 출력하시오. (k=5,10,20)**

# 싸이킷런을 이용한 K-겹 검증

[sklearn.model_selection.KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)를 이용하면 더 편하게 K-겹 검증 코드를 작성할 수 있습니다.

In [ ]:
from sklearn.model_selection import KFold

k = 4
num_epochs = 100
all_scores = []
kfold = KFold(n_splits=k)

for train_idx, val_idx in kfold.split(train_data):
    partial_train_data, val_data = train_data[train_idx], train_data[val_idx]
    partial_train_targets, val_targets = train_targets[train_idx], train_targets[val_idx]
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=16, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

print(all_scores)
print(np.mean(all_scores))